In [1]:
#basic imports
import pandas as pd
import numpy as np

#fitbit packages 
import fitbit
from python_fitbit import gather_keys_oauth2 as Oauth2

#time libraries
from datetime import datetime, timedelta
import pause

#data importing libraries
import ast
import glob
import json
import os

In [2]:
#set globals
#date I joined Fitbit
date_joined_fitbit = '2016-05-21'
today = datetime.now().date()
months = pd.date_range(start = date_joined_fitbit, end = today, freq = 'MS')
days = pd.date_range(start = date_joined_fitbit, end = today, freq = 'D')

In [8]:
def sleep_until_api_refresh():
    '''Sleep until the next time fitbit api refreshes'''
    #Identify next top of the hour from now
    #add 5 minutes in case fitbit is slow to update
    t = datetime.now()
    resume_time = t - timedelta(hours = -1, minutes = t.minute - 5, 
                                seconds = t.second, microseconds = t.microsecond)
    print(f'Sleeping until {str(resume_time.time())}')
    #pause until identified time
    pause.until(resume_time)
    
def activate_fitbit():
    '''
    Authorize the script to access my Fitbit data
    and return fitbit python object
    '''
    #import fitbit application credentials as a dictionary
    filepath = 'fitbit/fitbit_credentials.txt'
    with open(filepath, mode = 'r') as file:
        credentials = ast.literal_eval(file.read())
    
    #instantiate fitbit object
    client = fitbit.Fitbit(credentials['client_id'], 
                           credentials['client_secret'], 
                           oauth2=True, 
                           refresh_cb = True,
                           access_token=credentials['access_token'], 
                           refresh_token=credentials['refresh_token'])
    return client

def get_sleep(days):
    sleep_list = []
    for day in days:
        json = client.get_sleep(day)
        stats = pd.DataFrame(json['sleep'])
        summary = pd.DataFrame(json['summary'], index = [0])
        df = pd.concat([stats, summary], axis = 1)
        sleep_list.append(df)
        time.sleep(2)
    sleep = pd.concat(sleep_list, ignore_index = True, sort = False)
    sleep.to_csv('sleep.csv', index = False)
    return sleep

def get_weight(dates):
    '''
    Get historical weight data from Fitbit.
    Note: I log my weight using the MyFitnessPal app, which Fitbit
    downloads from. Because MyFitnessPal does not have a public API, I would
    have to download my weight statistics manually. This script allows me to
    programmatically access that data through Fitbit's connection to MyFitnessPal
    I also use this instead of the time series fitbit method because time series
    method imputes days where no weight was entered as the last known date. This way,
    I can manually interpolate the interim days' weight myself.
    '''
    #create empty list
    weight = []
    #iterate through period index of frequency month
    for date in dates:
        #create dataframe of weight data starting from the first of the month
        #to the end of the month
        df = pd.DataFrame(client.get_bodyweight(base_date = date, period = '1m')['weight'])
        #append to list
        weight.append(df)
    #concatenate into dataframe and export
    weight = pd.concat(weight, ignore_index = True, sort = False)
    weight.to_csv('fitbit/weight.csv', index = False)
    #return for console inspection
    return weight

def get_intraday_data(data):
    '''
    Get heart rata data at sub-minute granularity on workout days,
    updating for days I'm missing. 
    "heart" for heart rate, "calories" for calories
    '''
    
    '''identify days for which no heart rate data is downloaded'''
    #if a download folder doesn't exist, create it
    if os.path.exists(f"fitbit/{data}/") == False:
        os.makedirs(f"fitbit/{data}/")

    #import lifting data 
    lifts = pd.read_csv('lifts.csv', index_col = 'date')
    #get unique workout dates as datetime index as of the first date I joined fitbit
    lifting_days = pd.to_datetime(lifts.index).intersection(days).unique()
    
    #get list of heart rate files
    current_files = glob.glob(f'fitbit/{data}/*.json')
    
    #if there are no files in the folder, set the dates to download as 
    #the lifting days
    if len(current_files) == 0:
        dates_to_download = lifting_days
    
    #if there are some files downloaded, identify days for which there is no data
    else:
        #create index of filelist and replace all non-digit characters, leaving only the date
        downloaded_days = pd.Index(current_files).str.replace('\D+', '')
        #convert to datetime index 
        downloaded_days = pd.to_datetime(downloaded_days)

        #get dates in lifting days but not saved in a folder
        dates_to_download = lifting_days.difference(downloaded_days)

    '''download intraday data for all undownloaded days'''
    #set dictionary of level of granularity to pull for each data type
    if data == 'heart':
        interval = '1sec'
    else:
        interval = '1min'
    
    #download intraday data for all undownloaded days
    for day in dates_to_download:
        try:
            #convert date from timestamp to string
            day = str(day.date())
            #get intraday heart rate data at second granularity
            series = client.intraday_time_series(f'activities/{data}', 
                                                    base_date= day, 
                                                    detail_level= interval)
            #save to json
            with open(f'fitbit/{data}/{data}_{day}.json', 'w') as outfile:
                json.dump(series, outfile)

        except:
            #if the function hits an exception by hitting the fitbit rate limit, 
            #sleep for an hour
            #Fitbit's api has a rate limit of 150 requests per hour
            #which resets at the top of each hour, not necessarily an hour
            #after reaching thelimit
            print(str(Exception))
            sleep_until_api_refresh()
    print(f'{data} update complete')

def get_activities_report():
    '''Get list of activities logged in fitbit tracker'''
    beforeDate = str(today)
    while beforeDate > date_joined_fitbit:
        try:
            parameters = f'beforeDate={beforeDate}&offset=0&limit=20&sort=desc'
            activities = client.make_request(f'https://api.fitbit.com/1/user/-/activities/list.json&{parameters}')
            df = pd.DataFrame(activities['activities'])
            df['lastModified'] = pd.to_datetime(df.lastModified) 
            latest_date = str(df.lastModified.dt.date.max())
            beforeDate = str(df.lastModified.dt.date.min())
            df.to_csv(f'fitbit/activities/activities_{latest_date}.csv')
        except:
            sleep_until_api_refresh()

In [11]:
client = activate_fitbit()

In [ ]:
activity_indicators = ['heart', 'calories', 'steps', 'distance',
                       'floors', 'elevation']
for indicator in activity_indicators:
    get_intraday_data(indicator)

<class 'Exception'>
Sleeping until 23:05:00
<class 'Exception'>
Sleeping until 00:05:00
<class 'Exception'>
Sleeping until 01:05:00
<class 'Exception'>
Sleeping until 02:05:00
<class 'Exception'>
Sleeping until 03:05:00
<class 'Exception'>
Sleeping until 04:05:00


In [ ]:
get_activities_report()
#get_weight(months)

Todo: Write activities report function that identifies dates to pass over

Todo: Make sure that the sleep until refresh function doesn't end up skipping a day; make the loop go backwards